# **Spectral Clustering – eine empirische Untersuchung**

# Implementierung für Theorieteil


## Freie wissenschaftliche Arbeit zur Erlangung des akademischen Grades Bachelor of Science"

Studiengang: Wirtschaftsinformatik

**an der Wirtschaftswissenschaftlichen Fakultät der Universität Augsburg**

Lehrstuhl für Statistik

Eingereicht bei: Prof. Dr. Yarema Okhrin

Betreuerin:      Christine Distler (M. Sc.)

Vorgelegt von:

Adresse:
>
>
>



Augsburg, im März 2023



# Creating synthetic Dataset

In [ ]:
from sklearn import datasets
import matplotlib.pyplot as plt

In [ ]:
X, y = datasets.make_blobs(n_samples=10, centers=2, cluster_std=2, random_state=1)
# X, y = datasets.make_blobs(n_samples=21, centers=3, cluster_std=0.5, random_state=2)
# X, y = datasets.make_blobs(n_samples=40, centers=4, cluster_std=0.4, random_state=2)

plt.scatter(x=X[:, 0], y=X[:,1], s=30)
plt.show()

In [ ]:
X, y

Dieser Code stammt ursprünglich aus der Spectral Clustering Implementation von Dr. Yikun Zhang auf seinem [github Repository](https://github.com/zhangyk8/Spectral-Clustering/blob/master/spectral_clustering.py), wurde aber von mir modifiziert, um Graphen und Matrizen zurückzugeben.
 Zugriff: 05.12.2023

In [ ]:
from sklearn.datasets import make_moons
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial.distance import pdist, squareform
from sklearn.cluster import KMeans

In [ ]:
# Based on "A Tutorial on Spectral Clustering" written by Ulrike von Luxburg
def Spectral_Clustering(X, K=8, adj=True, metric='euclidean', sim_graph='fully_connect', sigma=1.0, knn=10, epsi=0.5,
                        normalized=1):
    """
    Input:
        X : [n_samples, n_samples] numpy array if adj=True, or, a [n_samples_a, n_features] array otherwise;

        K: int, The number of clusters;

        adj: boolean, Indicating whether the adjacency matrix is pre-computed. Default: True;

        metric: string, A parameter passing to "scipy.spatial.distance.pdist()" function for computing the adjacency
        matrix (deprecated if adj=True). Default: 'euclidean';

        sim_graph: string, Specifying the type of similarity graphs. Choices are ['fully_connect', 'eps_neighbor',
        'knn', 'mutual_knn']. Default: 'fully_connect';

        sigma: float, The variance for the Gaussian (aka RBF) kernel (Used when sim_graph='fully_connect'). Default: 1;

        knn: int, The number of neighbors used to construct k-Nearest Neighbor graphs (Used when sim_graph='knn'
        or 'mutual_knn'). Default: 10;

        epsi: float, A parameter controlling the connections between points (Used when sim_graph='eps_neighbor').
        Default: 0.5;

        normalized: int, 1: Random Walk normalized version; 2: Graph cut normalized version; other integer values:
        Unnormalized version. Default: 1.

    Output:
        sklearn.cluster class, Attributes:
            cluster_centers_ : array, [n_clusters, n_features], Coordinates of cluster centers in K-means;
            labels_ : Labels of each point;
            inertia_ : float, Sum of squared distances of samples to their closest cluster center in K-means;
            n_iter_ : int, Number of iterations run in K-means.
    """
    # Compute the adjacency matrix
    if not adj:
        Adj_mat = squareform(pdist(X, metric=metric))
    else:
        Adj_mat = X
    # Compute the weighted adjacency matrix based on the type of similarity graphs
    if sim_graph == 'fully_connect':
        W = np.exp(-np.square(Adj_mat) / (2 * sigma)**2)
    elif sim_graph == 'eps_neighbor':
        W = (Adj_mat <= epsi).astype('float64')
    elif sim_graph == 'knn':
        W = np.zeros(Adj_mat.shape)
        # Sort the adjacency matrx by rows and record the indices
        Adj_sort = np.argsort(Adj_mat, axis=1)
        # Set the weight (i,j) to 1 when either i or j is within the k-nearest neighbors of each other
        for i in range(Adj_sort.shape[0]):
            W[i, Adj_sort[i, :][:(knn + 1)]] = 1
    elif sim_graph == 'mutual_knn':
        W1 = np.zeros(Adj_mat.shape)
        # Sort the adjacency matrx by rows and record the indices
        Adj_sort = np.argsort(Adj_mat, axis=1)
        # Set the weight W1[i,j] to 0.5 when either i or j is within the k-nearest neighbors of each other (Flag)
        # Set the weight W1[i,j] to 1 when both i and j are within the k-nearest neighbors of each other
        for i in range(Adj_mat.shape[0]):
            for j in Adj_sort[i, :][:(knn + 1)]:
                if i == j:
                    W1[i, i] = 1
                elif W1[i, j] == 0 and W1[j, i] == 0:
                    W1[i, j] = 0.5
                else:
                    W1[i, j] = W1[j, i] = 1
        W = np.copy((W1 > 0.5).astype('float64'))

    else:
        raise ValueError(
            "The 'sim_graph' argument should be one of the strings, 'fully_connect', 'eps_neighbor', 'knn', or 'mutual_knn'!")

    # Compute the degree matrix and the unnormalized graph Laplacian
    D = np.diag(np.sum(W, axis=1))
    L = D - W

    # Compute the matrix with the first K eigenvectors as columns based on the normalized type of L
    if normalized == 1:  ## Random Walk normalized version
        # Compute the inverse of the diagonal matrix
        D_inv = np.diag(1 / np.diag(D))
        # Compute the eigenpairs of L_{rw}
        Lambdas, V = np.linalg.eig(np.dot(D_inv, L))
        # Sort the eigenvalues by their L2 norms and record the indices
        ind = np.argsort(np.linalg.norm(np.reshape(Lambdas, (1, len(Lambdas))), axis=0))
        V_K = np.real(V[:, ind[:K]])
    elif normalized == 2:  ## Graph cut normalized version
        # Compute the square root of the inverse of the diagonal matrix
        D_inv_sqrt = np.diag(1 / np.sqrt(np.diag(D)))
        # Compute the eigenpairs of L_{sym}
        Lambdas, V = np.linalg.eig(np.matmul(np.matmul(D_inv_sqrt, L), D_inv_sqrt))
        # Sort the eigenvalues by their L2 norms and record the indices
        ind = np.argsort(np.linalg.norm(np.reshape(Lambdas, (1, len(Lambdas))), axis=0))
        V_K = np.real(V[:, ind[:K]])
        if any(V_K.sum(axis=1) == 0):
            raise ValueError(
                "Can't normalize the matrix with the first K eigenvectors as columns! Perhaps the number of clusters K or the number of neighbors in k-NN is too small.")
        # Normalize the row sums to have norm 1
        V_K = V_K / np.reshape(np.linalg.norm(V_K, axis=1), (V_K.shape[0], 1))
    else:  ## Unnormalized version
        # Compute the eigenpairs of L
        Lambdas, V = np.linalg.eig(L)
        # Sort the eigenvalues by their L2 norms and record the indices
        ind = np.argsort(np.linalg.norm(np.reshape(Lambdas, (1, len(Lambdas))), axis=0))
        V_K = np.real(V[:, ind[:K]])

    # Conduct K-Means on the matrix with the first K eigenvectors as columns
    kmeans = KMeans(n_clusters=K, init='k-means++', random_state=0).fit(V_K)
    return kmeans, W, Adj_mat, Lambdas, V, V_K

show weighted adjancency matrices

In [ ]:
Lamb = None
experiment_parameters = {
    'K': 2,
    'sigma': 1,

    'epsilon': 3.5,

    'knn': 3, 
}
"""
experiment_parameters = {
    'K': 3,
    'sigma': 1,

    'epsilon': 3.5,

    'knn': 3, 
}"""

type_W = None
graphs = []
sim_graphs = ['fully_connect', 'eps_neighbor', 'knn', 'mutual_knn']

for graph_type in sim_graphs:
  kmeans_, W_, Adj_, Lambdas_, V_, V_K_ = Spectral_Clustering(X = X, K=experiment_parameters['K'], adj=False, sim_graph=graph_type, 
                                       knn=experiment_parameters['knn'], epsi=experiment_parameters['epsilon'])
  np.fill_diagonal(W_, 0)
  graphs.append(W_)
  
  if(graph_type == 'fully_connect'):
    print('not weigted Adjancency Martix (euclidean distance)')
    print(Adj_.round(2))
    print('Lambdas')
    print(Lambdas_.round(3))
    Lamb = Lambdas_
    print('V')
    print(V_.round(5))
    print('V_K')
    print(V_K_.round(2))
    print(kmeans_.labels_)
  

  print(graph_type)
  print(W_.round(3))

In [ ]:
# draw graphs
import matplotlib.pyplot as plt
import networkx as nx

In [ ]:
x_labels = {
    sim_graphs[0]: f"with Gaussian similarity function, sigma={experiment_parameters['sigma']}",
    sim_graphs[1]: f"epsilon={experiment_parameters['epsilon']}",
    sim_graphs[2]: f"k={experiment_parameters['knn']}",
    sim_graphs[3]: f"k={experiment_parameters['knn']}"
}

plt.figure(figsize=(10,10))
for i in range(len(graphs)):
  G = nx.from_numpy_array(graphs[i])
  plt.subplot(2, 2, i+1)
  nx.draw_networkx(G, pos=nx.circular_layout(G), font_color='white')
  plt.title(sim_graphs[i])
  plt.xlabel(x_labels[sim_graphs[i]])

In [ ]:
plt.figure(figsize=(8,8))
G = nx.from_numpy_array(graphs[0])
nx.draw_networkx(G, pos=nx.circular_layout(G), font_color='white', 
                 node_size=1500, font_size=16)
plt.xlabel(x_labels[sim_graphs[0]])
plt.show()

In [ ]:
"""
https://stackoverflow.com/questions/62935983/vary-thickness-of-edges-based-on-weight-in-networkx
ACCESS: 2023-02-15 20:06
"""

import networkx as nx
from matplotlib import pyplot as plt

G = nx.from_numpy_array(graphs[0])
widths = nx.get_edge_attributes(G, 'weight')
nodelist = G.nodes()

plt.figure(figsize=(8,8))

pos = nx.shell_layout(G)
nx.draw_networkx_nodes(G,pos,
                       nodelist=nodelist,
                       node_size=1000,
                       node_color='black',
                       alpha=1)
nx.draw_networkx_edges(G,pos,
                       edge_color='black',
                       alpha=1)
nx.draw_networkx_labels(G, pos=pos,
                        labels=dict(zip(nodelist,nodelist)),
                        font_color='white')
ED = G.edges(data=True)
for u,v,w in ED:
  w['weight'] = round(w['weight'],3)
nx.draw_networkx_edge_labels(G,pos=pos, edge_labels=nx.get_edge_attributes(G, 'weight'))

plt.box(False)
plt.show()

# Untersuchen der Gausschen Ähnlichkeitsfunktion

In [ ]:
import numpy as np
from scipy.spatial.distance import pdist, squareform
from sklearn import datasets

In [ ]:
X2, y2 = datasets.make_blobs(n_samples=15, centers=2,
                           random_state=0)
Adj_mat = squareform(pdist(X2, metric='euclidean'))
def gaussian(mat, sigma):
    return np.exp(-np.square(mat) / (2 * sigma)**2)



In [ ]:
Adj_mat

In [ ]:
gaussian(Adj_mat, 0.1)

In [ ]:
gaussian(Adj_mat, 1.2)

In [ ]:
# plot the data from Adj_mat at x and gaussian(Adj_mat, 1.2) at y
# in the same plot plot the data from Adj_mat at x and gaussian(Adj_mat, 0.1) at y
# plot with legend
import matplotlib.pyplot as plt
plt.figure(figsize=(10,10))
plt.scatter(Adj_mat, gaussian(Adj_mat, 1.5))
plt.scatter(Adj_mat, gaussian(Adj_mat, 0.8))
plt.scatter(Adj_mat, gaussian(Adj_mat, 0.3))
plt.xlabel('Adjacency Matrix')
plt.ylabel('Gaussian Similarity Function')
plt.legend(['sigma=1.2', 'sigma=0.6', 'sigma=0.1'])
plt.show()

# Random Walk Laplacian $L_{rw}$


In [ ]:
Adj_mat = squareform(pdist(X, metric='euclidean'))
W = np.exp(-np.square(Adj_mat) / (2 * 0.5)**2)
D = np.diag(np.sum(W, axis=1))
L = D - W
D_inv = np.diag(1 / np.diag(D))
L_rw = np.dot(D_inv, L)
print(L_rw.round(2))

In [ ]:
Lambdas, V = np.linalg.eig(np.dot(D_inv, L))
print(V.shape)
print(Lambdas)

In [ ]:
#@title Eigengap Heuristic

# np.argsort(np.diff(Lamb))[::-1][:5]
print(np.argsort(np.diff(Lamb)))
print(f'Eigenvalues: {Lamb}')
def maxDiffIndex(array): 
  # Sortiere das Array
  array = np.sort(array) 
  # Initialisiere eine Variable für den größtmöglichen Index 
  max_diff_index = 0
  # Initialisiere eine Variable für die größtmögliche Differenz 
  max_diff = 0
  
  # Iteriere über das Array 
  for i in range(1, len(array)): 
    # Berechne die Differenz zwischen zwei aufeinander folgenden Elementen
    diff = array[i] - array[i-1]
    # Prüfe, ob die Differenz größer als die vorherige ist
    if diff > max_diff:
      # Wenn ja, speichere diesen Index und die Differenz in den Variablen
      max_diff_index = i
      max_diff = diff
  
  # Gib den größtmöglichen Index und die Differenz
  return max_diff_index, max_diff
print(maxDiffIndex(Lamb))

In [ ]:
plt.scatter([i for i in range(len(Lamb))], np.sort(Lamb))